In [2]:
from collections import (Counter,
                         defaultdict,
                         deque,
                         )

Task:
Write a function top_k_frequent(nums: list[int], k: int) -> list[int] that returns the k most frequent numbers in nums.
If two numbers have the same frequency, return the smaller number first.

In [3]:
def top_k_frequent(nums: list[float], k: int) -> list[tuple[float, int]]:
    frequencies = list(Counter(nums).most_common())

    # exploit the fact that sort algorithm in python is stable
    # in case of tie it return the first occurrence
    frequencies.sort(key=lambda t: t[0])
    frequencies.sort(key=lambda t: t[1], reverse=True)
    top_k = frequencies[:k]
    return [num for num, _ in top_k]

assert top_k_frequent([1, 2, 3, 2, 2, 1], k=2) == [2, 1]
assert top_k_frequent([1, 2, 3, 2, 2, 3, 3], k=1) == [2]
assert top_k_frequent([1, 2, 3, 2, 2, 3, 3, 1, 1], k=2) == [1, 2]
assert top_k_frequent([1., 2., 3., 2., 2, 3., 3., 1., 1], k=2) == [1, 2]

Task: Given a list of numbers, return the first element that appears exactly once.
If no such element exists, return None.

In [4]:
def first_unique(nums: list[int]) -> int | None:
    unique_nums = filter(lambda d: d[1] == 1, Counter(nums).items())
    try:
        return next(unique_nums)[0]
    except StopIteration:
        return None

assert first_unique([1, 2, 3, 2, 4]) == 1
assert first_unique([1, 2, 3, 2, 4, 3, 1, 5]) == 4
assert first_unique([1, 2, 3, 2, 3, 2, 1]) is None

Task: Given a string, return the first character that appears exactly once, ignoring case.
Return None if all characters are repeating.

In [5]:
# Using default dict (retain the insertion order)
def first_unique_char(string: str) -> str | None:
    counter = defaultdict(int)
    lowercased_string = string.casefold()
    for char in lowercased_string:
        counter[char] += 1

    for orig_char, lower_char in zip(string, lowercased_string):
        if counter[lower_char] == 1:
            return orig_char

    return None

assert first_unique_char("Swiss") == "w"
assert first_unique_char("aAbBABac") == "c"
assert first_unique_char("xX") == None
assert first_unique_char("cabBd") == "c"

In [7]:
# Return element in decreasing order and in case of tie retain the insertion order
def first_unique_char(string: str) -> str | None:
    lowercased_string = string.casefold()
    counter = Counter(lowercased_string)
    for orig_char, lower_char in zip(string, lowercased_string):
        if counter[lower_char] == 1:
            return orig_char

    return None

assert first_unique_char("Swiss") == "w"
assert first_unique_char("aAbBABac") == "c"
assert first_unique_char("xX") == None
assert first_unique_char("cabBd") == "c"

Task: Given a list of integers nums and an integer k, return a list of the maximum values in every sliding window of size k.

In [8]:
# Simplest approach but there is a better solution in terms of performance.
def sliding_window_max(nums: list[float| int], k: int) -> list[float| int]:
    if len(nums) <= k:
        return [max(nums)]

    max_numbers_per_window = []
    for i in range(len(nums) - k + 1):
        max_numbers_per_window.append(max(nums[i:i + k]))

    return max_numbers_per_window

assert sliding_window_max([1,3,-1,-3,5,3,6,7], k=3) == [3,3,5,5,6,7], f"{sliding_window_max([1,3,-1,-3,5,3,6,7], k=3)}"
assert sliding_window_max([1], k=1) == [1]
assert sliding_window_max([9, 11], k=2) == [11]

In [9]:
def sliding_window_max(nums: list[int], k: int) -> list[int]:
    """
        1. Remove numbers not belonging to the window
        2. If the current number is greater than the previous remove the previous number,
        since is not a candidate for max anymore.
    """
    if not nums or k <= 0:
        return []

    result = []
    # stores indices of elements in decreasing order
    dq = deque()
    for i, num in enumerate(nums):
        # 1. Remove indices that are out of this window
        # (e.g. for k=2, when i=2 the minimum index in the deque must be 1)
        while dq and dq[0] <= i - k:
            dq.popleft()
        # 2. Remove smaller numbers from the right (they are useless)
        # if the previous number is lower than the current number then it is not a candidate
        while dq and nums[dq[-1]] < num:
            dq.pop()
        # 3. Add current index
        dq.append(i)  # <-- the last index in the deque point to the previous number of the nums

        # 4. The leftmost element is the max of the window
        if i >= k - 1:
            result.append(nums[dq[0]])

    return result

assert sliding_window_max([1,3,-1,-3,5,3,6,7], k=3) == [3,3,5,5,6,7]
assert sliding_window_max([1], k=1) == [1]
assert sliding_window_max([9, 11], k=2) == [11]

Task: Given a list of numbers and an integer k, rotate the list to the right by k steps.
Use a deque to do this efficiently.

In [13]:
def rotate_array(nums: list, k: int) -> list:
    if len(nums) == 0:
        return []

    dq = deque(nums, maxlen=len(nums))
    for _ in range(k):
        dq.appendleft(dq.pop())

    return list(dq)

assert rotate_array([1,2,3], k=2) == [2, 3, 1]
assert rotate_array([1, 2, 3, 4], k=5) == [4, 1, 2, 3]
assert rotate_array([], k=5) == []


Task: Given a list of numbers and a window size k, return a list of the median for each sliding window.
Use a deque to maintain the current window efficiently.
The median is:
 - The middle value if k is odd
 - The average of the two middle values if k is even

In [42]:
# Simplest but I have to sort the window every time....
def sliding_window_median(nums: list[int | float], k: int) -> list[int | float]:
    def _calculate_median(sorted_nums: list[int | float]) -> float:
        n = len(sorted_nums)
        mid = n // 2
        if n % 2:
            return sorted_nums[mid]
        return (sorted_nums[mid - 1] + sorted_nums[mid]) / 2

    if k <= 0 or not nums:
        return []

    results = []
    window = deque(nums[:k], maxlen=k)

    for num in nums[k:]:
        results.append(_calculate_median(sorted(window)))
        window.append(num)

    # last window's median
    results.append(_calculate_median(sorted(window)))
    return results


assert sliding_window_median([1, 3, -1, -3, 5, 3, 6, 7], k=3) == [1, -1, -1, 3, 5, 6]
assert sliding_window_median([1, 4, 2, 3], k=4) == [2.5]
assert sliding_window_median([1], k=1) == [1]
assert sliding_window_median([1, 2], k=1) == [1, 2]

In [48]:
import bisect

l = [1, 2, 2, 3, 3, 6]
# return the leftmost position which maintain the list sorted
print(bisect.bisect_left(l, 3))
# return the rightmost position which maintain the list sorted
print(bisect.bisect_right(l, 3))

3
5


In [52]:
def sliding_window_median(nums: list[int | float], k: int) -> list:
    def _calculate_median(sorted_nums: list[int | float]) -> float:
        if k % 2 == 1:
            mid = len(sorted_nums) // 2
            return sorted_nums[mid]
        else:
            mid = len(sorted_nums) // 2
            return (sorted_nums[mid - 1] + sorted_nums[mid]) / 2


    if not nums or k <= 0:
        return []

    sorted_window = sorted(nums[:k])
    dq = deque(nums[:k])
    results = list()
    results.append(_calculate_median(sorted_window))

    for num in nums[k:]:
        # add the new number
        dq.append(num)
        bisect.insort(sorted_window, num)
        # remove the leftmost
        removed = dq.popleft()
        idx = bisect.bisect_left(sorted_window, removed)
        sorted_window.pop(idx)

        results.append(_calculate_median(sorted_window))

    return results


assert sliding_window_median([1, 3, -1, -3, 5, 3, 6, 7], k=3) == [1, -1, -1, 3, 5, 6]
assert sliding_window_median([1, 4, 2, 3], k=4) == [2.5]
assert sliding_window_median([1], k=1) == [1]
assert sliding_window_median([1, 2], k=1) == [1, 2]

In [69]:
# ! pip install sortedcontainers
from sortedcontainers import SortedList

def sliding_window_median(nums: list[int | float], k: int) -> list[float]:
    def _calculate_median(sorted_nums: list[int | float]) -> float:
        if k % 2 == 1:
            mid = len(sorted_nums) // 2
            return sorted_nums[mid]
        else:
            mid = len(sorted_nums) // 2
            return (sorted_nums[mid - 1] + sorted_nums[mid]) / 2

    if not nums or k <= 0:
        return []

    window = SortedList(nums[:k])
    results = [_calculate_median(window)]

    for i in range(k, len(nums)):
        # slide window
        window.remove(nums[i - k])
        window.add(nums[i])
        results.append(_calculate_median(window))

    return results

assert sliding_window_median([1, 3, -1, -3, 5, 3, 6, 7], k=3) == [1, -1, -1, 3, 5, 6]
assert sliding_window_median([1, 4, 2, 3], k=4) == [2.5]
assert sliding_window_median([1], k=1) == [1]
assert sliding_window_median([1, 2], k=1) == [1, 2]

Task: Implement a class RecentCounter that counts recent requests.
Description:
- The class has a method ping(t: int) -> int where t is a timestamp in milliseconds.
- Each call to ping(t) adds a new request at time t.
- The method should return the number of requests in the last 3000 milliseconds (inclusive of the current one).


In [78]:
class RecentCounter(deque):
    def ping(self, timestamp):
        while self and self[0] + 3000 <= timestamp:
            self.popleft()

        self.append(timestamp)
        return len(self)


rc = RecentCounter()
assert rc.ping(1) == 1
assert rc.ping(2) == 2
assert rc.ping(3) == 3
assert rc.ping(100) == 4
assert rc.ping(3001) == 4
assert rc.ping(3002) == 4
assert rc.ping(3010) == 4


Task: Given a binary array nums (only 0s and 1s). You can flip at most one 0 to 1.
Return the length of the longest contiguous subarray of 1s after at most one flip.

In [88]:
def longest_ones(nums: list[int]) -> int:
    zeros_indices = deque()
    left = 0
    max_len = 0

    for right, num in enumerate(nums):
        if num == 0:
            zeros_indices.append(right)

        if len(zeros_indices) > 1:
            left = zeros_indices.popleft() + 1

        max_len = max(max_len, right - left + 1)

    return max_len

assert longest_ones([1, 1]) == 2
assert longest_ones([1, 1, 0, 1]) == 4
assert longest_ones([1, 1, 1, 0, 0, 1, 0, 1, 1, 1]) == 5
